#### 1. Observability
Monitoring:
- Input-Output prompts.
- Token usage.
- Latency.
- Cost.
- Runtime errors.

#### 2. Metrics
Numerically measure the accuracy and quality. <br>
Also helps in thinking of best case scenarios

#### 3. Optimizations
Finetuning LMs or prompts to improve metrics.